# PyCity Schools Analysis:
### Author: Ashley Paillet
### Data Trends: There does not seem to be a positive relationship between a school's spending per student and its student's performance. It appears that the relationship between School's Spending Range per Student is negative. The schools that have the highest performance spend less per student.
### Data Trends: The data indicates that the students of the charter schools have more suspense than their counterparts at district schools. 8 of 10 top performing schools are charter schools. There is a large gap in the overall passing rate between students at charter schools and those in district schools.


In [1]:
# Setup
import pandas as pd
from pathlib import Path

# File load
schools_import_data = Path('Resources/schools_complete.csv')
students_import_data = Path('Resources/students_complete.csv')

# Store data in Pandas DataFrame & rename headers
schools_df = pd.read_csv(schools_import_data)
schools_df = schools_df.rename( columns = {
    'school_name' : 'School',
    'type' : 'Type',
    'size' : 'Size',
    'budget' : 'Budget'})

students_df = pd.read_csv(students_import_data)
students_df = students_df.rename( columns = {
    'student_name' : 'Student Name',
    'gender' : 'Gender',
    'school_name' : 'School',
    'grade' : 'Grade',
    'reading_score' : 'Reading Score',
    'math_score' : 'Math Score'})

# Combine data
combined_school_df = pd.merge(students_df, schools_df, how = 'left', on = ['School', 'School'])

combined_school_df.head()

,Student ID,Student Name,Gender,Grade,School,Reading Score,Math Score,School ID,Type,Size,Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Check for missing data
combined_school_df.count()

Student ID       39170
Student Name     39170
Gender           39170
Grade            39170
School           39170
Reading Score    39170
Math Score       39170
School ID        39170
Type             39170
Size             39170
Budget           39170
dtype: int64

In [3]:
# Number of unique schools
school_sum = len(combined_school_df['School'].unique())
print(f'There are {school_sum} schools total')

There are 15 schools total


In [4]:
# Number of students
# len(combined_school_df['student_name'].unique())
# len(students_df['student_name'].unique())
# 32715 --Why is this wrong??

student_sum = combined_school_df['Student Name'].count()
print(f'There are {student_sum} students total')

There are 39170 students total


In [5]:
# Total budget
budget_sum = schools_df['Budget'].sum()
print(f'There is a total of $ {budget_sum} budgeted')

There is a total of $ 24649428 budgeted


In [6]:
# Average math score
math_avg = round(combined_school_df['Math Score'].mean(),2)
print(f'The average math score is {math_avg}')

The average math score is 78.99


In [7]:
# Average reading score
reading_avg = round(combined_school_df['Reading Score'].mean(),2)
print(f'The average reading score is {reading_avg}')

The average reading score is 81.88


In [8]:
# Percentage who passed math
# passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
# passing_math_percentage = passing_math_count / float(student_count) * 100
# passing_math_percentage
math_pass_sum = combined_school_df[(combined_school_df['Math Score'] >= 70)].count()['Student Name']
math_pass_rate = round((math_pass_sum / float(student_sum)) * 100, 2)
print(f'{math_pass_rate} % of students passed math')

74.98 % of students passed math


In [9]:
# Percentage who passed reading
reading_pass_sum = combined_school_df[(combined_school_df['Reading Score'] >= 70)].count()['Student Name']
reading_pass_rate = round((reading_pass_sum / float(student_sum)) * 100, 2)
print(f'{reading_pass_rate} % of students passed reading')

85.81 % of students passed reading


In [10]:
# Percentage who passed both math and reading
math_reading_sum = combined_school_df[
    (combined_school_df['Math Score'] >= 70) & (combined_school_df['Reading Score'] >= 70)].count()['Student Name']
math_reading_rate = round(((math_reading_sum / float(student_sum)) * 100), 2)
print(f'{math_reading_rate} % of students passed both math and reading')

65.17 % of students passed both math and reading


In [11]:
# District Summary
district_summary_df = pd.DataFrame({
    'Total Schools' : [school_sum],
    'Total Students' : [student_sum],
    'Total Budget' : [budget_sum],
    'Average Math Score' : [math_avg],
    'Average Reading Score' : [reading_avg],
    '% Passing Math' : [math_pass_rate],
    '% Passing Reading' : [reading_pass_rate],
    '% Overall Passing' : [math_reading_rate]})

# Formatting
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].map("{:.1f}%".format)
district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].map("{:.1f}%".format)
district_summary_df['% Overall Passing'] = district_summary_df['% Overall Passing'].map("{:.1f}%".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,75.0%,85.8%,65.2%


## School Summary

In [12]:
school_types = schools_df.groupby('School')['Type'].sum()
# print(school_types)


In [13]:
# Calculate the total student count per school
student_body_school = combined_school_df.groupby('School')['Size'].mean()
# student_body_school

In [14]:
# Calculate the total school budget and per capita spending per school
school_budget = combined_school_df.groupby('School')['Budget'].mean()
budget_per_capita = round(school_budget / student_body_school, 2)
# budget_per_capita

In [15]:
# Average test score by school
math_score_school = round(combined_school_df.groupby('School')['Math Score'].mean(), 2)
reading_score_school = round(combined_school_df.groupby('School')['Reading Score'].mean(), 2)
# print(f'{math_score_school} \n \n {reading_score_school}')

In [16]:
# Number of students passing math by school
passing_math_school = combined_school_df[(combined_school_df['Math Score'] >= 70)].groupby('School')['Math Score'].count()
# passing_math_school

In [17]:
# Number of students passing reading by school
passing_reading_school = combined_school_df[(combined_school_df['Reading Score'] >= 70)].groupby('School')['Reading Score'].count()
# passing_reading_school

In [18]:
# Number of students passing math and reading by school
passing_both_school = combined_school_df[
    (combined_school_df['Math Score'] >= 70) & (combined_school_df['Reading Score'] >= 70)].groupby('School')['Student Name'].count()
# passing_both_school

In [19]:
# Passing rates: math, reading, both
math_rate_school = round((passing_math_school / student_body_school) * 100, 2)

reading_rate_school = round((passing_reading_school / student_body_school) * 100, 2)

both_rate_school = round((passing_both_school / student_body_school) * 100, 2)

# print(f'{math_rate_school} \n \n {reading_rate_school} \n \n {both_rate_school}')

In [20]:
school_summary_df = pd.DataFrame({
    'School Type': school_types, 
    'Total Students': student_body_school, 
    'Total School Budget': school_budget, 
    'Per Student Budget': budget_per_capita,
    'Average Math Score': math_score_school, 
    'Average Reading Score': reading_score_school, 
    '% Passing Math': math_rate_school, 
    '% Passing Reading': reading_rate_school, 
    '% Overall Passing': both_rate_school})

# Subsequent data frames made from this frame do not work if the formatting is added because it converts them from floats and ints to objects. Making a copy to add formatting to each.
school_copy_df = school_summary_df.copy(deep=True)

# Formatting
school_copy_df['Total Students'] = school_copy_df['Total Students'].map('{:,}'.format)
school_copy_df['Total School Budget'] = school_copy_df['Total School Budget'].map('${:,.2f}'.format)
school_copy_df['Per Student Budget'] = school_copy_df['Per Student Budget'].map('${:,.2f}'.format)
school_copy_df['% Passing Math'] = school_copy_df['% Passing Math'].map('{:.1f}%'.format)
school_copy_df['% Passing Reading'] = school_copy_df['% Passing Reading'].map('{:.1f}%'.format)
school_copy_df['% Overall Passing'] = school_copy_df['% Overall Passing'].map('{:.1f}%'.format)
school_copy_df.index.name = None

school_copy_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976.0","$3,124,928.00",$628.00,77.05,81.03,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,66.0%,80.7%,53.2%
Ford High School,District,"2,739.0","$1,763,916.00",$644.00,77.10,80.75,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.4%,97.1%,90.6%
Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.8%,80.9%,53.5%
Holden High School,Charter,427.0,"$248,087.00",$581.00,83.80,83.81,92.5%,96.2%,89.2%
Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.7%,81.3%,53.5%
Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.1%,81.2%,53.5%
Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.6%,96.0%,90.5%


## Highest-Performing Schools (by % Overall Passing)

In [21]:
# % Overall Passing in descending order
top_schools_df = school_summary_df.sort_values(by = ['% Overall Passing'], ascending = False)

# Formatting
top_schools_df['Total Students'] = top_schools_df['Total Students'].map('{:,}'.format)
top_schools_df['Total School Budget'] = top_schools_df['Total School Budget'].map('${:,.2f}'.format)
top_schools_df['Per Student Budget'] = top_schools_df['Per Student Budget'].map('${:,.2f}'.format)
top_schools_df['% Passing Math'] = top_schools_df['% Passing Math'].map('{:.1f}%'.format)
top_schools_df['% Passing Reading'] = top_schools_df['% Passing Reading'].map('{:.1f}%'.format)
top_schools_df['% Overall Passing'] = top_schools_df['% Overall Passing'].map('{:.1f}%'.format)
top_schools_df.index.name = None

top_schools_df.head(10)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635.0","$1,043,130.00",$638.00,83.42,83.85,93.3%,97.3%,91.0%
Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283.0","$1,319,574.00",$578.00,83.27,83.99,93.9%,96.5%,90.6%
Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.6%,96.0%,90.5%
Wright High School,Charter,"1,800.0","$1,049,400.00",$583.00,83.68,83.96,93.3%,96.6%,90.3%
Shelton High School,Charter,"1,761.0","$1,056,600.00",$600.00,83.36,83.73,93.9%,95.8%,89.9%
Holden High School,Charter,427.0,"$248,087.00",$581.00,83.80,83.81,92.5%,96.2%,89.2%
Bailey High School,District,"4,976.0","$3,124,928.00",$628.00,77.05,81.03,66.7%,81.9%,54.6%
Ford High School,District,"2,739.0","$1,763,916.00",$644.00,77.10,80.75,68.3%,79.3%,54.3%


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# % Overall Passing in ascending order
bottom_schools_df = school_summary_df.sort_values(by = ['% Overall Passing'], ascending = True)

# Formatting
bottom_schools_df['Total Students'] = bottom_schools_df['Total Students'].map('{:,}'.format)
bottom_schools_df['Total School Budget'] = bottom_schools_df['Total School Budget'].map('${:,.2f}'.format)
bottom_schools_df['Per Student Budget'] = bottom_schools_df['Per Student Budget'].map('${:,.2f}'.format)
bottom_schools_df['% Passing Math'] = bottom_schools_df['% Passing Math'].map('{:.1f}%'.format)
bottom_schools_df['% Passing Reading'] = bottom_schools_df['% Passing Reading'].map('{:.1f}%'.format)
bottom_schools_df['% Overall Passing'] = bottom_schools_df['% Overall Passing'].map('{:.1f}%'.format)
bottom_schools_df.index.name = None

bottom_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999.0","$2,547,363.00",$637.00,76.84,80.74,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,66.0%,80.7%,53.2%
Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.8%,80.9%,53.5%
Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.1%,81.2%,53.5%


## Math Scores by Grade

In [23]:
# Separate the data by grade
ninth_gr = combined_school_df[(combined_school_df['Grade'] == '9th')]
tenth_gr = combined_school_df[(combined_school_df['Grade'] == '10th')]
eleventh_gr = combined_school_df[(combined_school_df['Grade'] == '11th')]
twelfth_gr = combined_school_df[(combined_school_df['Grade'] == '12th')]

# Mean math score by school, by grade

math_ninth = round(ninth_gr.groupby('School')['Math Score'].mean(), 2)
math_tenth = round(tenth_gr.groupby('School')['Math Score'].mean(), 2)
math_eleventh = round(eleventh_gr.groupby('School')['Math Score'].mean(), 2)
math_twelfth = round(twelfth_gr.groupby('School')['Math Score'].mean(), 2)

# Data Frame: Math Scores By Grade
math_by_gr = pd.DataFrame({
    '9th': math_ninth, 
    '10th': math_tenth, 
    '11th': math_eleventh, 
    '12th': math_twelfth})

math_by_gr

,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [24]:
# Mean reading score by school, by grade

reading_ninth = round(ninth_gr.groupby('School')['Reading Score'].mean(), 2)
reading_tenth = round(tenth_gr.groupby('School')['Reading Score'].mean(), 2)
reading_eleventh = round(eleventh_gr.groupby('School')['Reading Score'].mean(), 2)
reading_twelfth = round(twelfth_gr.groupby('School')['Reading Score'].mean(), 2)

# Data Frame: Math Scores By Grade
reading_by_gr = pd.DataFrame({
    '9th': reading_ninth, 
    '10th': reading_tenth, 
    '11th': reading_eleventh, 
    '12th': reading_twelfth})

reading_by_gr

,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [25]:
# School summary copy df
school_spending_df = school_summary_df.copy(deep=True)

# Establish bins 
spending_bins = [0, 585, 630, 645, 680]
bin_label = ['<$585', '$585-630', '$630-645', '$645-680']
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget'], spending_bins, labels = bin_label )
spending_range_student = school_spending_df['Spending Ranges (Per Student)']

spending_copy_df = school_spending_df.copy(deep=True)
# Formatting
spending_copy_df['Total Students'] = spending_copy_df['Total Students'].map('{:,}'.format)
spending_copy_df['Total School Budget'] = spending_copy_df['Total School Budget'].map('${:,.2f}'.format)
spending_copy_df['Per Student Budget'] = spending_copy_df['Per Student Budget'].map('${:,.2f}'.format)
spending_copy_df['% Passing Math'] = spending_copy_df['% Passing Math'].map('{:.1f}%'.format)
spending_copy_df['% Passing Reading'] = spending_copy_df['% Passing Reading'].map('{:.1f}%'.format)
spending_copy_df['% Overall Passing'] = spending_copy_df['% Overall Passing'].map('{:.1f}%'.format)
spending_copy_df.index.name = None

spending_copy_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,"4,976.0","$3,124,928.00",$628.00,77.05,81.03,66.7%,81.9%,54.6%,$585-630
Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.1%,97.0%,91.3%,<$585
Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,66.0%,80.7%,53.2%,$630-645
Ford High School,District,"2,739.0","$1,763,916.00",$644.00,77.10,80.75,68.3%,79.3%,54.3%,$630-645
Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.4%,97.1%,90.6%,$585-630
Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.8%,80.9%,53.5%,$645-680
Holden High School,Charter,427.0,"$248,087.00",$581.00,83.80,83.81,92.5%,96.2%,89.2%,<$585
Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.7%,81.3%,53.5%,$645-680
Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.1%,81.2%,53.5%,$645-680
Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.6%,96.0%,90.5%,$585-630


In [26]:
# Scores by spending ranges
spending_math_scores = round(school_spending_df.groupby(['Spending Ranges (Per Student)'])['Average Math Score'].mean(), 2)
spending_reading_scores = round(school_spending_df.groupby(['Spending Ranges (Per Student)'])['Average Reading Score'].mean(), 2)
spending_passing_math = round(school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Passing Math'].mean(), 2)
spending_passing_reading = round(school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Passing Reading'].mean(), 2)
overall_passing_spending = round(school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Overall Passing'].mean(), 2)

In [27]:
# New DataFrame
spending_summary_df = pd.DataFrame({
'Average Math Score': spending_math_scores,
'Average Reading Score': spending_reading_scores,
'% Passing Math': spending_passing_math,
'% Passing Reading': spending_passing_reading,
'% Overall Passing': overall_passing_spending})

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [28]:
# Copying old data frame to form new
school_size_df = school_summary_df.copy(deep = True)
# Creating bins and labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
school_size_df['School Size'] = pd.cut(school_size_df['Total Students'], size_bins, labels = size_labels)

size_copy_df = school_size_df.copy(deep = True)
# Formatting
size_copy_df['Total Students'] = size_copy_df['Total Students'].map('{:,}'.format)
size_copy_df['Total School Budget'] = size_copy_df['Total School Budget'].map('${:,.2f}'.format)
size_copy_df['Per Student Budget'] = size_copy_df['Per Student Budget'].map('${:,.2f}'.format)
size_copy_df['% Passing Math'] = size_copy_df['% Passing Math'].map('{:.1f}%'.format)
size_copy_df['% Passing Reading'] = size_copy_df['% Passing Reading'].map('{:.1f}%'.format)
size_copy_df['% Overall Passing'] = size_copy_df['% Overall Passing'].map('{:.1f}%'.format)
size_copy_df.index.name = None
size_copy_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,"4,976.0","$3,124,928.00",$628.00,77.05,81.03,66.7%,81.9%,54.6%,Large (2000-5000)
Cabrera High School,Charter,"1,858.0","$1,081,356.00",$582.00,83.06,83.98,94.1%,97.0%,91.3%,Medium (1000-2000)
Figueroa High School,District,"2,949.0","$1,884,411.00",$639.00,76.71,81.16,66.0%,80.7%,53.2%,Large (2000-5000)
Ford High School,District,"2,739.0","$1,763,916.00",$644.00,77.10,80.75,68.3%,79.3%,54.3%,Large (2000-5000)
Griffin High School,Charter,"1,468.0","$917,500.00",$625.00,83.35,83.82,93.4%,97.1%,90.6%,Medium (1000-2000)
Hernandez High School,District,"4,635.0","$3,022,020.00",$652.00,77.29,80.93,66.8%,80.9%,53.5%,Large (2000-5000)
Holden High School,Charter,427.0,"$248,087.00",$581.00,83.80,83.81,92.5%,96.2%,89.2%,Small (<1000)
Huang High School,District,"2,917.0","$1,910,635.00",$655.00,76.63,81.18,65.7%,81.3%,53.5%,Large (2000-5000)
Johnson High School,District,"4,761.0","$3,094,650.00",$650.00,77.07,80.97,66.1%,81.2%,53.5%,Large (2000-5000)
Pena High School,Charter,962.0,"$585,858.00",$609.00,83.84,84.04,94.6%,96.0%,90.5%,Small (<1000)


In [29]:
# Scores by size
size_math_scores = round(school_size_df.groupby(['School Size'])['Average Math Score'].mean(), 2)
size_reading_scores = round(school_size_df.groupby(['School Size'])['Average Reading Score'].mean(), 2)
size_passing_math = round(school_size_df.groupby(['School Size'])['% Passing Math'].mean(), 2).map('{:.1f}%'.format)
size_passing_reading = round(school_size_df.groupby(['School Size'])['% Passing Reading'].mean(), 2).map('{:.1f}%'.format)
size_overall_passing = round(school_size_df.groupby(['School Size'])['% Overall Passing'].mean(), 2).map('{:.1f}%'.format)

In [30]:
# Data Frame for school performance based on school size (small, medium, or large)
size_summary_df = pd.DataFrame({
    'Average Math Score': size_math_scores,
    'Average Reading Score': size_reading_scores,
    '% Passing Math': size_passing_math,
    '% Passing Reading': size_passing_reading,
    '% Overall Passing': size_overall_passing})

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.5%,96.1%,89.9%
Medium (1000-2000),83.37,83.87,93.6%,96.8%,90.6%
Large (2000-5000),77.74,81.34,70.0%,82.8%,58.3%


## Scores by School Type

In [31]:
# Copy data frame
type_df = school_summary_df.copy(deep = True)

# Group the per_school_summary DataFrame by "School Type" and average the results.
avg_math_type = round(type_df.groupby(["School Type"])["Average Math Score"].mean(), 2)
avg_reading_type = round(type_df.groupby(["School Type"])["Average Reading Score"].mean(), 2)
avg_passing_math_type = round(type_df.groupby(["School Type"])["% Passing Math"].mean(), 2).map('{:.1f}%'.format)
avg_passing_reading_type = round(type_df.groupby(["School Type"])["% Passing Reading"].mean(), 2).map('{:.1f}%'.format)
avg_overall_type = round(type_df.groupby(["School Type"])["% Overall Passing"].mean(), 2).map('{:.1f}%'.format)

In [32]:
# Summary Data Frame
type_summary_df = pd.DataFrame({
    'Average Math Score': avg_math_type,
    'Average Reading Score': avg_reading_type,
    '% Passing Math': avg_passing_math_type,
    '% Passing Reading': avg_passing_reading_type,
    '% Overall Passing': avg_overall_type})

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.6%,96.6%,90.4%
District,76.96,80.97,66.5%,80.8%,53.7%
